In [ ]:
import numpy as np
import time


# --- Funções Lógicas ---


def sign_extend_24_to_32(value):
    """
    Simula a Extensão com Sinal (Sign Extension) de 24 bits para 32 bits.
    """
    MSB_24_BIT_MASK = 0x800000 # Bit 23
   
    value_24bit = value & 0xFFFFFF
   
    if value_24bit & MSB_24_BIT_MASK:
        # Negativo: subtrai 2^24 para obter a representação correta de 32 bits (C2)
        return np.int32(value_24bit - 0x1000000)
    else:
        # Positivo
        return np.int32(value_24bit)


def mux_2_1_resultado_32bit(alu_result_32bit, acc_result_24bit, mux_res_sel):
    """
    Simula o Multiplexador 2:1 de Resultado com saída de 32 bits.
    """
    if mux_res_sel == 0:
        return alu_result_32bit
   
    elif mux_res_sel == 1:
        return sign_extend_24_to_32(acc_result_24bit)
       
    else:
        raise ValueError("mux_res_sel deve ser 0 ou 1.")


# --- Função de Execução e Timing ---


def run_test_case(test_id, alu_res, acc_res, sel, test_desc, expected_res):
    """Executa o teste, mede o tempo e exibe o resultado e verificação."""
   
    start_time = time.perf_counter()
   
    # Execução da função do MUX
    resultado = mux_2_1_resultado_32bit(alu_res, acc_res, sel)
   
    end_time = time.perf_counter()
    duration = (end_time - start_time) * 1e6 # Converte para microssegundos (µs)
   
    print(f"\n--- Teste {test_id}: {test_desc} (Sel={sel}) ---")
    print(f"  Entrada ACC (24-bit): {hex(acc_res & 0xFFFFFF)}")
    print(f"  Resultado Saída: {hex(resultado)}")
    print(f"  Valor Esperado:  {hex(expected_res)}")
    print(f"  Tempo de Execução (Python): {duration:.3f} µs")
    print(f"  Verificação: {'✅ PASSOU' if resultado == expected_res else '❌ FALHOU'}")
   
    return resultado


# --- Variáveis de Limites (Casos de Borda e Overflow/Underflow) ---


print("## Testbench MUX 2:1 - Testes Unificados (Seleção, Limites e Sign Extension) ##")


# Limites do ALU (32 bits)
ALU_MAX_POS = np.int32(0x7FFFFFFF)              
ALU_MIN_NEG = np.int32(-2147483648)              


# Limites do Acumulador (24 bits)
ACC_MAX_POS_VAL = 0x007FFFFF                    
ACC_MIN_NEG_VAL = 0x00800000                    
ACC_NEG_ONE_VAL = 0x00FFFFFF                    


# Valores Típicos de Teste (para seleção normal)
ALU_TYPICAL = np.int32(0x12345678)
ACC_TYPICAL = np.int32(0x00AABBCC)


# Variável dummy (não usada na seleção)
DUMMY = np.int32(-1431655766)


# --- Pre-cálculo do Valor Esperado para o Teste 2 (GOLDEN REFERENCE) ---
# Usa a função de Sign Extension para garantir o valor correto para 0x00AABBCC
EXPECTED_2_CORRECTED = sign_extend_24_to_32(ACC_TYPICAL)


# --- Definição e Execução dos Casos de Teste ---


test_cases = [
    # 1. CASOS DE SELEÇÃO NORMAL (Funcionalidade Básica)
    (1, ALU_TYPICAL, ACC_TYPICAL, 0, "Seleção ALU (Caso Típico)", ALU_TYPICAL),
   
    # 2. CORRIGIDO: Seleção Acumulador (Caso Típico)
    # Valor esperado obtido diretamente da função de referência para evitar erros manuais.
    (2, ALU_TYPICAL, ACC_TYPICAL, 1, "Seleção Acumulador (Caso Típico - FINALMENTE CORRIGIDO)", EXPECTED_2_CORRECTED),
   
    # 3. CASOS DE BORDA DO ALU (32 bits)
    (3, ALU_MAX_POS, DUMMY, 0, "ALU: Limite Máximo Positivo", ALU_MAX_POS),
    (4, ALU_MIN_NEG, DUMMY, 0, "ALU: Limite Mínimo Negativo (Underflow)", ALU_MIN_NEG),
   
    # 5. CASOS DE BORDA DO ACUMULADOR (24 bits) COM SIGN EXTENSION
    (5, DUMMY, ACC_MAX_POS_VAL, 1, "ACC: Limite Máximo Positivo", np.int32(ACC_MAX_POS_VAL)),
   
    # 6. Max Negativo (0x00800000 -> 0xFF800000) - Teste de Underflow
    (6, DUMMY, ACC_MIN_NEG_VAL, 1, "ACC: Limite Mínimo Negativo (Sign Ext.)", np.int32(-8388608)),
   
    # 7. Valor -1 (0x00FFFFFF -> 0xFFFFFFFF) - Teste de Sinal
    (7, DUMMY, ACC_NEG_ONE_VAL, 1, "ACC: Valor -1 (Sign Ext.)", np.int32(-1)),
]


# Executa todos os testes
for test_id, alu, acc, sel, desc, expected in test_cases:
    run_test_case(test_id, alu, acc, sel, desc, expected)
